# Create dice objects to be used in simple dice game simulations

In [13]:
import numpy as np

class Die:
    '''
    Dice have a number of sides and, unless in motion, always have one face up.  
    
    Cocked dice are ignored, and motion/trajectories are assumed to be a random choice from the number of sides.
    '''
    
    def __init__(self,sides):
        self.sides = sides
        self.face_up = np.random.choice([i+1 for i in range(self.sides)])
        
    def roll(self):
        self.face_up = np.random.choice([i+1 for i in range(self.sides)])
        
        return self.face_up

In [14]:
die = Die(sides=6)

In [3]:
die.face_up

2

In [4]:
die.roll()

5

In [5]:
die.face_up

5

In [138]:
import numpy as np
from collections import Counter

class Dice_Game:
    '''
    This simple multiplayer dice game is the Beebe family's variation of a popular dice game, we just call it "Dice".  
    
    Instructions:
    
    Players determine who goes first, and proceed taking turns in an order (typically clockwise around a table).
    
    Each player, in turn, proceeds by first shaking all six dice, 
    and proceeding along a decision tree of subsequent rolls.
    This tree ends in either a zero score, where a shake results in no dice count for points, 
    or the player "keeps" a number of dice which count for points.
    
    The point value of dice are determined per shake, 
    and point values do not change when mixed with dice kept from previous rolls.
    
    COUNTERS:
    
    One: 100
    Five: 50
    
    THREE-OF-A-KIND:
    Ones: 1000
    Sixes: 600
    Fives: 500
    Fours: 400
    Threes: 300
    Twos: 200
    
    STRAIGHT (1,2,3,4,5,6): 2500
    
    FLUSH (ALL-OF-A-KIND): Instant Win.
    
    On the occasion where all dice rolled are "counters", the dice must all be brought back and shook again.
    The turn proceeds as usual, adding the new shake points to the previous all-counter results.
    
    
    '''
    
    def __init__(self, players, num_dice=6):
        self.player_names = ['Player_{}'.format(name + 1) for name in [j for j in range(players)]]
        self.players = [self.Player(name=player) for player in self.player_names]
        self.dice = [self.Die(sides=6) for die in [i for i in range(num_dice)]]
        self.play_to = 7500
        self.winner = False
        self.turn = 0
        
        
    def start():
        self.score = score
        
    
    def shake(self,dice_to_roll,Player):
        shook_dice = [die.roll() for die in dice_to_roll]
        print('I,', Player.name, 'shook the dice:', [i.face_up for i in dice_to_roll])
        #return shook_dice
        return dice_to_roll

        
        
    def player_take_turn(self,Player):
        still_to_shake = self.dice.copy()
        kept = []
        Player.turn_shakes = 0
        #print('length of self.dice', len(self.dice))
        while len(still_to_shake) > 0:
            if Player.turn_shakes == 0:
                shake = self.shake(still_to_shake,Player)
                shake_faces = [i.face_up for i in shake]
                #print('shake:',shake_faces)
                Player.turn_shakes += 1
                counters = self.detect_keepable(shake)
                kept, still_to_shake = self.keep_dice(shake, counters)
                #print('objects still to shake:',still_to_shake)
                print('still to shake faces:', [i.face_up for i in still_to_shake])
                Player.learning_history.append(kept)
            elif kept != [] and Player.turn_shakes > 0:
                print("keep 'em rollin'!")
                shake = self.shake(still_to_shake,Player)
                shake_faces = [i.face_up for i in shake]
                print('shake:',shake_faces)
                Player.turn_shakes += 1
                counters = self.detect_keepable(shake)
                kept, still_to_shake = self.keep_dice(shake, counters)
                print('still to shake:',still_to_shake)
                print('still to shake faces:', [i.face_up for i in still_to_shake])
                Player.learning_history.append(kept)
            elif kept == [] and Player.turn_shakes > 0:
                print("you got nuthin:'",shake_faces, ". next player's turn.")
                break
        
        Player.turn += 1
        
        
        
    def detect_keepable(self, shook_dice):
        
        '''
        Method which sorts through the shook dice and returns a list (of lists) of potentially keepable dice.
        
        For example, if the dice objects shook show [1,2,5,4,4,4] detect_keepable will return: [[4,4,4],1,5]
        
        '''
        
        print('BEGIN DETECTING KEEPABLES')
        
        #print('shook dice:', shook_dice)
        
        dice_faces = [i.face_up for i in shook_dice]
        #print('dice faces', dice_faces)
        
        three_of_a_kinds = [i for i in Counter(dice_faces) if Counter(dice_faces)[i] >= 3]
        print('three_of_a_kinds:', three_of_a_kinds)
        
        keepable_three_of_a_kinds = []
        for set_of_three in three_of_a_kinds:
            print('set_of_three',set_of_three)
            toak = []
            for i in shook_dice:
                if i.face_up == set_of_three and len(toak) <= 2:
                    toak.append(i)
            keepable_three_of_a_kinds.append(toak)
            #keepable_three_of_a_kinds.append([i for i in shook_dice if i.face_up == set_of_three])
            #print('appended keepable three of a kind objects:', keepable_three_of_a_kinds)

        keepable = keepable_three_of_a_kinds
        #print('keepable objects after adding three of a kinds:', keepable)
        print('keepable dice faces after adding three of a kinds:', [j.face_up for i in keepable for j in i])

        
        for i in shook_dice:
            if i.face_up == 1 or i.face_up == 5:
                keepable.append(i)
        #print('keepable objects after adding ones and fives:', keepable)
        print('keepable dice faces after adding ones and fives:', [i.face_up if type(i) != list else [j.face_up for j in i] for i in keepable])
                

        if set(dice_faces) == set([1,2,3,4,5,6]):
            keepable.append(shook_dice)
            #print('keepable objects after adding straight:', keepable)
            print('keepable dice faces after adding straight:', [i.face_up if type(i) != list else [j.face_up for j in i] for i in keepable])

        print('total keepable list of dice faces:', [i.face_up if type(i) != list else [j.face_up for j in i] for i in keepable])
        return keepable
    
    
    def keep_dice(self, shook_dice, keepable_list):
        '''
        Method for choosing which dice to keep.  
        
        This will in particular need to be interfaced for agent to learn.
        
        '''
        
        print('BEGIN KEEPING DICE')
        
        #print('keepable object list:', keepable_list)
        print('keepable dice faces list:', [i.face_up if type(i) != list else [j.face_up for j in i] for i in keepable_list])
        print('len keepable_list:', len(keepable_list))
        if len(keepable_list) is 0 or 1:
            kept_dice = keepable_list
        else:
            kept_dice = [np.random.choice(keepable_list)]
        #print('kept dice objects:', kept_dice)
        print('kept dice faces:', [i.face_up if type(i) != list else [j.face_up for j in i] for i in kept_dice])
        

        for i in kept_dice:
            #print('object i in kept_dice:', i)
            if type(i) is list:
                print('i in kept_dice is list:', [j.face_up for j in i])
                for j in i:
                    print('j object in list i:', j)
                    print('j.face_up in list i:', j.face_up)
                    shook_dice.remove(j)
                    print('shook_dice removed j:',shook_dice)
            elif i in shook_dice:
                shook_dice.remove(i)

        dice_still_to_shake = shook_dice

        return kept_dice, dice_still_to_shake

    
    
    class Die:
        '''
        Dice have a number of sides and, unless in motion, always have one face up.  

        Cocked dice are ignored, and motion/trajectories are assumed to be a random choice from the number of sides.
        '''

        def __init__(self,sides):
            self.sides = sides
            self.face_up = np.random.choice([i+1 for i in range(self.sides)])

        def roll(self):
            self.face_up = np.random.choice([i+1 for i in range(self.sides)])

            return self.face_up
    
    # Players as subclass, called in Dice_Game init
    class Player:
        '''
        Each player is equipped with several attributes, mainly for statistics and for future agent-based learning.
        '''
    
        def __init__(self,name):
            self.name = name
            self.turn = 0
            self.score = 0
            self.learning_history = []
            self.turn_shakes = 0
            self.game_shakes = 0

            


In [139]:
game = Dice_Game(players=3)

In [140]:
game.play_to

7500

In [141]:
print([i.face_up for i in game.dice])

[6, 2, 6, 2, 5, 5]


In [142]:
game.players[0].name

'Player_1'

In [143]:
shake = game.shake(game.dice,game.players[0])

I, Player_1 shook the dice: [5, 6, 5, 5, 1, 5]


In [145]:
game.player_take_turn(game.players[0])

I, Player_1 shook the dice: [4, 1, 5, 3, 3, 4]
BEGIN DETECTING KEEPABLES
three_of_a_kinds: []
keepable dice faces after adding three of a kinds: []
keepable dice faces after adding ones and fives: [1, 5]
total keepable list of dice faces: [1, 5]
BEGIN KEEPING DICE
keepable dice faces list: [1, 5]
len keepable_list: 2
kept dice faces: [1, 5]
still to shake: [<__main__.Dice_Game.Die object at 0x1182fe690>, <__main__.Dice_Game.Die object at 0x1182fe810>, <__main__.Dice_Game.Die object at 0x1182fe8d0>, <__main__.Dice_Game.Die object at 0x1182feb50>]
still to shake faces: [4, 3, 3, 4]
keep 'em rollin'!
I, Player_1 shook the dice: [4, 2, 6, 5]
shake: [4, 2, 6, 5]
BEGIN DETECTING KEEPABLES
three_of_a_kinds: []
keepable dice faces after adding three of a kinds: []
keepable dice faces after adding ones and fives: [5]
total keepable list of dice faces: [5]
BEGIN KEEPING DICE
keepable dice faces list: [5]
len keepable_list: 1
kept dice faces: [5]
still to shake: [<__main__.Dice_Game.Die object at

In [52]:
game.players[0].turn

1

In [ ]:
def kept_dice_to_potential_score(kept_dice):
    
    
    
    return potential_score

In [146]:
def stay_or_go(kept,Player):
    
    return decision